# Clasificación de Tumores Cerebrales con CNN (Entrenamiento y Validación 80/20)

Este notebook está preparado para el desarrollo y entrenamiento de una CNN sobre imágenes MRI. Incluye:
- Separación automática 80/20 para entrenamiento y validación
- Visualización de datos
- Definición y entrenamiento del modelo
- Guardado automático de checkpoints cada N épocas
- Tamaño de imagen: 512x512 píxeles

In [1]:
# Importar bibliotecas necesarias
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ReduceLROnPlateau,
    ModelCheckpoint,
    TensorBoard,
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Configuración de semilla para reproducibilidad
np.random.seed(42)
tf.random.set_seed(42)

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Configuración de rutas
BASE_DIR = Path("../total/archive/Augmented")
MODEL_DIR = Path("../models")
print(f"BASE_DIR: {BASE_DIR}")

# Detectar clases automáticamente
classes = [d.name for d in BASE_DIR.iterdir() if d.is_dir()]
print(f"Clases detectadas: {classes}")
num_classes = len(classes)

# Parámetros
IMG_SIZE = 512
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.001
DROPOUT_RATE = 0.5
L2_REG = 0.0001

BASE_DIR: ..\total\archive\Augmented
Clases detectadas: ['glioma', 'meningioma', 'no_tumor', 'pituitary']


In [2]:
# Generador de datos con separación 80/20 (entrenamiento/validación)
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.9, 1.1],
    validation_split=0.2,
)

train_generator = train_datagen.flow_from_directory(
    str(BASE_DIR),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    color_mode="rgb",
)

val_generator = train_datagen.flow_from_directory(
    str(BASE_DIR),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False,
    color_mode="rgb",
)

Found 48092 images belonging to 4 classes.
Found 12020 images belonging to 4 classes.


In [3]:
# Definir arquitectura de la CNN
def create_brain_tumor_model(
    input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=num_classes
):
    model = Sequential(
        [
            Conv2D(
                32,
                (3, 3),
                activation="relu",
                padding="same",
                input_shape=input_shape,
                kernel_regularizer=l2(L2_REG),
            ),
            BatchNormalization(),
            Conv2D(
                32,
                (3, 3),
                activation="relu",
                padding="same",
                kernel_regularizer=l2(L2_REG),
            ),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Dropout(DROPOUT_RATE / 2),
            Conv2D(
                64,
                (3, 3),
                activation="relu",
                padding="same",
                kernel_regularizer=l2(L2_REG),
            ),
            BatchNormalization(),
            Conv2D(
                64,
                (3, 3),
                activation="relu",
                padding="same",
                kernel_regularizer=l2(L2_REG),
            ),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Dropout(DROPOUT_RATE),
            Conv2D(
                128,
                (3, 3),
                activation="relu",
                padding="same",
                kernel_regularizer=l2(L2_REG),
            ),
            BatchNormalization(),
            Conv2D(
                128,
                (3, 3),
                activation="relu",
                padding="same",
                kernel_regularizer=l2(L2_REG),
            ),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Dropout(DROPOUT_RATE),
            Flatten(),
            Dense(256, activation="relu", kernel_regularizer=l2(L2_REG)),
            BatchNormalization(),
            Dropout(DROPOUT_RATE),
            Dense(num_classes, activation="softmax"),
        ]
    )
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.AUC(name="auc")],
    )
    return model


In [4]:
import glob
from tensorflow.keras.models import load_model

# Cargar el último modelo .keras guardado en MODEL_DIR o crear uno nuevo si no existe
model_files = sorted(glob.glob(str(MODEL_DIR / '*.keras')))
if model_files:
    last_model = model_files[-1]
    model = load_model(last_model)
    print(f"Cargando modelo desde: {last_model}")
else:
    model = create_brain_tumor_model()

model.summary()

Cargando modelo desde: ..\models\best_model.keras


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 512, 512, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512, 512, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 512, 512, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512, 512, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 128, 128, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 524288)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │   134,217,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 403,523,694 (1.50 GB)

 Trainable params: 134,507,428 (513.11 MB)

 Non-trainable params: 1,408 (5.50 KB)

 Optimizer params: 269,014,858 (1.00 GB)

In [5]:
# Callbacks: EarlyStopping, ReduceLROnPlateau, y ModelCheckpoint cada N épocas
callbacks = [
    EarlyStopping(
        monitor="val_auc", patience=15, mode="max", restore_best_weights=True
    ),
    ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=8),
    ModelCheckpoint(
        "../models/best_model.keras", monitor="val_auc", save_best_only=True, mode="max"
    ),
    TensorBoard(log_dir="./logs"),
]

In [6]:
# Entrenamiento del modelo con validación
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks,

)

c:\Users\dev-max\Documents\GitHub\inteligencia_de_negocios\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
 122/1503 ━━━━━━━━━━━━━━━━━━━━ 4:11:32 11s/step - accuracy: 0.7452 - auc: 0.9297 - loss: 1.3018

KeyboardInterrupt: 

In [ ]:
# Evaluar el modelo en el conjunto de validación
val_loss, val_acc, val_auc = model.evaluate(val_generator, workers=4, use_multiprocessing=True)
print(f"\nPrecisión Validación: {val_acc:.4f}")
print(f"AUC Validación: {val_auc:.4f}")

# Guardar el modelo final
final_model_path = MODEL_DIR / 'brain_tumor_cnn_model_final.h5'
model.save(str(final_model_path))
print(f"Modelo final guardado en: {final_model_path}")

In [ ]:
# Visualización de la curva de pérdida y precisión
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Precisión del modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc='lower right')
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Pérdida del modelo')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc='upper right')
plt.tight_layout()
plt.show()

**Notas:**
- El entrenamiento se realiza con separación 80/20 para entrenamiento y validación.
- Se guardan checkpoints automáticos cada N épocas y el modelo final al terminar.
- Puedes ajustar los hiperparámetros y arquitectura según tus necesidades.

# Aumentación de Datos para Imágenes MRI

Antes de entrenar la CNN, es recomendable aplicar aumentación para mejorar la robustez del modelo. A continuación se muestra cómo realizar aumentación usando `ImageDataGenerator` de Keras y visualizar ejemplos.

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

AUG_IMG_SIZE = 512
AUG_BATCH_SIZE = 4
AUG_DIR = Path("../total/archive/Augmented")

aug_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.9, 1.1],
    fill_mode='nearest',
)

aug_generator = aug_datagen.flow_from_directory(
    str(AUG_DIR),
    target_size=(AUG_IMG_SIZE, AUG_IMG_SIZE),
    batch_size=AUG_BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
    color_mode="rgb",
)

images, labels = next(aug_generator)
plt.figure(figsize=(10, 3))
for i in range(AUG_BATCH_SIZE):
    plt.subplot(1, AUG_BATCH_SIZE, i+1)
    plt.imshow(images[i])
    plt.axis('off')
plt.suptitle('Ejemplos de aumentación')
plt.show()